In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# Define constants
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 10  # Change this to the number of classes in your dataset
EPOCHS = 10

In [ ]:
# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Load the dataset
train_generator = train_datagen.flow_from_directory(
    '/kaggle/input/miniimagenet',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

In [ ]:
validation_generator = validation_datagen.flow_from_directory(
    '/kaggle/input/miniimagenet',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

In [ ]:
# Load the pre-trained VGG19 model (without the top classification layer)
base_model = VGG19(
    weights='imagenet',
    include_top=False,
    input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)
)

In [ ]:
# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

In [ ]:
# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    steps_per_epoch=len(train_generator),
    validation_steps=len(validation_generator)
)

In [ ]:
# Save the trained model
model.save('vgg19_transfer_learning.h5')

In [ ]:
# Load your trained model
model = tf.keras.models.load_model('vgg19_transfer_learning.h5')

# Use the validation generator to make predictions
validation_generator.reset()
y_pred = model.predict(validation_generator, steps=len(validation_generator), verbose=1)
y_true = validation_generator.classes

In [ ]:
# Convert predicted probabilities to class labels
y_pred_classes = np.argmax(y_pred, axis=1)

In [ ]:
# Calculate accuracy
accuracy = np.mean(y_pred_classes == y_true)
print(f'Accuracy: {accuracy * 100:.2f}%')

In [ ]:
# Calculate confusion matrix
confusion = confusion_matrix(y_true, y_pred_classes)
print('Confusion Matrix:')
print(confusion)

In [ ]:
# Generate a classification report
class_names = list(validation_generator.class_indices.keys())
classification_rep = classification_report(y_true, y_pred_classes, target_names=class_names)
print('Classification Report:')
print(classification_rep)